In [18]:
# 📊 Optimized MACD Trading Strategy for 1-Minute Data with Fixes for Over-Trading & Loss Prevention

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Download real-time 1-minute data for a stock (S&P 500 E-mini Futures or another ticker)
ticker = "ES=F"  # S&P 500 E-mini Futures
df = yf.download(ticker, period="5d", interval="1m")  # Increased period to 5 days

# Ensure proper indexing
df = df.reset_index()
df.rename(columns={"Datetime": "Date"}, inplace=True)
df.set_index("Date", inplace=True)

# Debugging: Check if data was downloaded correctly
print(df.head())
print(df.tail())

# Optimized MACD Calculation for 1-minute data (Adjusted to Reduce Over-Trading)
df["6_EMA"] = df["Close"].ewm(span=5, adjust=False).mean()
df["13_EMA"] = df["Close"].ewm(span=10, adjust=False).mean()
df["MACD"] = df["6_EMA"] - df["13_EMA"]
df["Signal_Line"] = df["MACD"].ewm(span=4, adjust=False).mean()

# Trading Signals
df["Position"] = 0
df.loc[df["MACD"] > df["Signal_Line"], "Position"] = 1  # Buy Signal
df.loc[df["MACD"] < df["Signal_Line"], "Position"] = -1  # Sell Signal
df["Trade_Signal"] = df["Position"].diff()

# Fix trade signal values to ensure correct buy/sell triggers
df["Trade_Signal"] = df["Trade_Signal"].replace({2: 1, -2: -1})

# Debugging: Check number of trade signals
print("Trade Signal Counts:")
print(df["Trade_Signal"].value_counts())

# Stop-Loss & Take-Profit Adjustments to Avoid Over-Trading
stop_loss_pct = 0.03  # Increased stop-loss to 3% to prevent small fluctuations from stopping trades
take_profit_pct = 0.07  # Increased take-profit to 7% to maximize profit potential
minimum_profit_threshold = 2.00  # Minimum $2 profit per trade to avoid micro trades
min_price_movement = 2.00  # Minimum price change before executing new trade

# Extended Cooldown Period to Reduce Excessive Trading
cooldown_period = pd.Timedelta(minutes=10)  # Prevent over-trading by increasing wait time

# Remove trend filter to allow more trades
def trend_filter(index, trade_type):
    return True  # Disabled trend filter

# Backtesting Strategy with Dynamic Trade Sizing, Loss Prevention & Cooldown Mechanism
initial_cash = 10000  # Start capital
cash = initial_cash
shares = 0
portfolio_value = []
entry_price = 0
trade_log = []

total_trades = 0
total_profit = 0
last_trade_time = None

# Adjust Trade Sizing to Avoid Large Drawdowns
def calculate_trade_size(price, cash_available):
    return max(1, int(cash_available / (price * 20)))  # Allocates ~5% of capital per trade

for i in range(len(df)):
    price = df["Close"].iloc[i]
    
    # Cooldown logic to prevent excessive trading
    if last_trade_time is not None and (df.index[i] - last_trade_time) < cooldown_period:
        continue  # Skip this trade if within cooldown
    
    if df["Trade_Signal"].iloc[i] == 1:  # Buy Signal
        shares = calculate_trade_size(price, cash)
        if shares > 0 and cash >= shares * price:  # Prevent negative balance
            cash -= shares * price
            entry_price = price  # Store entry price
            trade_log.append(f"🚀 TRADE EXECUTED: BUY {shares} shares at {price:.2f} on {df.index[i]}")
            total_trades += 1
            last_trade_time = df.index[i]  # Update last trade time

    elif df["Trade_Signal"].iloc[i] == -1 and shares > 0:  # Sell Signal
        if abs(price - entry_price) >= min_price_movement:  # Ensure minimum price movement
            cash += shares * price
            profit = (price - entry_price) * shares
            if profit >= minimum_profit_threshold:  # Ensure minimum profit threshold
                total_profit += profit
                trade_log.append(f"📉 TRADE EXECUTED: SELL at {price:.2f} on {df.index[i]} | Profit: ${profit:.2f}")
            shares = 0
            entry_price = 0
            last_trade_time = df.index[i]  # Update last trade time
    
    # Stop-Loss & Take-Profit Logic
    if shares > 0:
        if price < entry_price * (1 - stop_loss_pct):  # Stop-Loss Trigger
            cash += shares * price
            profit = (price - entry_price) * shares
            total_profit += profit
            trade_log.append(f"❌ STOP-LOSS at {price:.2f} on {df.index[i]} | Loss: ${profit:.2f}")
            shares = 0
            entry_price = 0
            last_trade_time = df.index[i]
        elif price > entry_price * (1 + take_profit_pct):  # Take-Profit Trigger
            cash += shares * price
            profit = (price - entry_price) * shares
            total_profit += profit
            trade_log.append(f"✅ TAKE-PROFIT at {price:.2f} on {df.index[i]} | Profit: ${profit:.2f}")
            shares = 0
            entry_price = 0
            last_trade_time = df.index[i]
    
    portfolio_value.append(cash + shares * price)

# Ensure df is trimmed to match portfolio_value length before assignment
df = df.iloc[:len(portfolio_value)]
df["MACD_Strategy_Value"] = portfolio_value

# Buy & Hold Benchmark
df["Buy_Hold_Value"] = initial_cash * (df["Close"] / df["Close"].iloc[0])

# 📊 Print Clear Summary & Conclusion
print("\n" + "="*60)
print("📈 **Final Trading Summary**")
print("="*60)
print(f"🔹 Total Trades Executed: {total_trades}")
print(f"🔹 Total Profit from Trading: ${total_profit:,.2f}")

print("\n📊 **Comparison of MACD Trading vs. Buy & Hold**")
print(f"🔹 MACD Strategy Final Value: ${df['MACD_Strategy_Value'].iloc[-1]:,.2f}")
print(f"🔹 Buy & Hold Final Value: ${df['Buy_Hold_Value'].iloc[-1]:,.2f}")

if df['MACD_Strategy_Value'].iloc[-1] > df['Buy_Hold_Value'].iloc[-1]:
    print("✅ **MACD Trading Outperformed the Market!**")
else:
    print("❌ **MACD Trading Underperformed the Market!**")
print("="*60)

# Print trade logs
print("\nTrade Log:")
for trade in trade_log:
    print(trade)


[*********************100%%**********************]  1 of 1 completed

                              Open     High      Low    Close  Adj Close  \
Date                                                                       
2025-02-10 00:00:00-05:00  6064.25  6065.00  6064.25  6064.50    6064.50   
2025-02-10 00:01:00-05:00  6064.25  6064.75  6064.25  6064.50    6064.50   
2025-02-10 00:02:00-05:00  6064.50  6065.25  6064.50  6065.25    6065.25   
2025-02-10 00:03:00-05:00  6065.00  6065.25  6065.00  6065.25    6065.25   
2025-02-10 00:04:00-05:00  6065.00  6066.00  6065.00  6065.75    6065.75   

                           Volume  
Date                               
2025-02-10 00:00:00-05:00       0  
2025-02-10 00:01:00-05:00      74  
2025-02-10 00:02:00-05:00      62  
2025-02-10 00:03:00-05:00      72  
2025-02-10 00:04:00-05:00     102  
                              Open     High      Low    Close  Adj Close  \
Date                                                                       
2025-02-14 04:50:00-05:00  6136.25  6136.25  6133.25  6133.50  